In [118]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.metrics import accuracy_score, f1_score, classification_report,roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
from sklearn.ensemble import StackingClassifier
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.svm import SVC

from sklearn.cluster import DBSCAN


In [119]:
data = pd.read_csv('../ani_data.csv')
df = pd.DataFrame(data)


# print("total 0 :", (df['target']==0).sum()) -> 298
# print("total 1 :", (df['target']==1).sum()) -> 83


df = df.drop(['snowfall (cm)','windspeed_10m (km/h)'], axis=1)
# df.sample()

In [120]:
#removing outliers

#value = mean + 3* std

#for temperature 
# upper_threshold = df['temperature_2m (Â°C)'].mean() + (3*df['temperature_2m (Â°C)'].std())
# lower_threshold = df['temperature_2m (Â°C)'].mean() - (3*df['temperature_2m (Â°C)'].std())

# p75 = df['temperature_2m (Â°C)'].quantile(.75)
# p25 = df['temperature_2m (Â°C)'].quantile(.25)



# iqr = p75-p25
# upper_threshold = p75 + (1.5 * iqr)
# lower_threshold = p75 - (1.5 * iqr)
# print(upper_threshold, lower_threshold)
# 15.737499999999999 -15.987499999999999
# print((df['temperature_2m (Â°C)'] > upper_threshold).sum())
# df['temperature_2m (Â°C)'] = np.where(df['temperature_2m (Â°C)'] > upper_threshold, upper_threshold, df['temperature_2m (Â°C)'])
# df['temperature_2m (Â°C)'] = np.where(df['temperature_2m (Â°C)'] < lower_threshold, lower_threshold, df['temperature_2m (Â°C)'])

# sns.boxplot(df['temperature_2m (Â°C)'])
# plt.show()
# sns.boxplot(df['windspeed_100m (km/h)'])
# plt.show()

In [121]:
X = df.drop(['time','target'
             ],axis=1)
y = df['target']

mas = MinMaxScaler()
X_train_scaled = mas.fit_transform(X)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.25, random_state=42)


In [123]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [141]:
#https://www.tutorialspoint.com/scikit_learn/scikit_learn_anomaly_detection.htm
clf = DBSCAN(eps=3, min_samples=6).fit(X)
clf.fit(X_train_resampled)

y_pred = clf.fit_predict(X_test)
accuracy = accuracy_score(y_test, y_pred)  # Fix variable name
print(f1_score(y_test, y_pred, average='macro')*100
)
print(f"Accuracy: {accuracy}")

45.76271186440678
Accuracy: 0.84375


In [ ]:
rf = RandomForestClassifier(max_depth=2, n_estimators=10,
    random_state=42)
xgb = XGBClassifier(random_state=42,n_estimators=300,learning_rate=0.3,max_depth=6 )
svm = SVC()

estimators = [('xgboost', xgb),('svm',svm), ('random forest', rf)]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

stacking_clf.fit(X_train_resampled, y_train_resampled)


StackingClassifier(estimators=[('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learn...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=6,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=300, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('svm', SVC()),
                               ('random forest',
                                RandomForestClassifier(max_depth=2,
                                                       n_estimators=10,
                                                       random_state=42))],
                   final_estimator=LogisticRegression())

In [ ]:
y_pred = stacking_clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


report = classification_report(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred)
print("Accuracy:", round(accuracy * 100, 2), "%")
print("F1 Score:", round(f1 * 100, 2), "%")
print("Classification Report:\n", report)
print("ROC Report:\n", roc)

Accuracy: 88.54 %
F1 Score: 64.52 %
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        81
           1       0.62      0.67      0.65        15

    accuracy                           0.89        96
   macro avg       0.78      0.80      0.79        96
weighted avg       0.89      0.89      0.89        96

ROC Report:
 0.7962962962962963


In [ ]:
X = df.drop(['time','target'
             ],axis=1)
y = df['target']

mas = MinMaxScaler()
X_train_scaled = mas.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.25, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

svm =XGBClassifier(random_state=42,n_estimators=300,learning_rate=0.3,max_depth=6 )

svm.fit(X_train_resampled, y_train_resampled)
y_pred = svm.predict(X_test)  # Fix variable name
accuracy = accuracy_score(y_test, y_pred)  # Fix variable name

print(f"Accuracy: {accuracy}")
print("f1",f1_score(y_test, y_pred))

print(df.corr()['target'])



Accuracy: 0.8333333333333334
f1 0.3846153846153846
temperature_2m (Â°C)       0.025533
relativehumidity_2m (%)   -0.049383
cloudcover_mid (%)         0.073262
windspeed_100m (km/h)      0.002059
winddirection_10m (Â°)     0.026890
winddirection_100m (Â°)    0.028825
target                     1.000000
Name: target, dtype: float64


/tmp/ipykernel_10992/1819211616.py:23: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.corr()['target'])
